In [1]:
%load_ext autoreload
%autoreload 2
from src.datasets.mimic import make_mimic
from src.models.vision_transformer import PatchEmbed
import torch
if torch.cuda.is_available():
    device="cuda:0"
    torch.cuda.set_device(device=device)
else:
    device="cpu"


Get Data

In [25]:
from src.masks.multiblock import MaskCollator
import numpy as np

dataset, data_loader = make_mimic(None, 1, collator=MaskCollator(input_size=(12,5000), patch_size=(1,100)))
iterator = enumerate(data_loader)
idx, (data,masks_enc,masks_pred) = next(iterator)


In [28]:
data = data.to(device)
masks_enc = [u.to(device) for u in masks_enc]
masks_pred = [u.to(device) for u in masks_pred]
print(masks_enc[0].shape, masks_pred[0].shape)
print(masks_pred)

torch.Size([1, 77]) torch.Size([1, 143])
[tensor([[ 31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  81,
          82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93, 131, 132,
         133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 181, 182, 183,
         184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 231, 232, 233, 234,
         235, 236, 237, 238, 239, 240, 241, 242, 243, 281, 282, 283, 284, 285,
         286, 287, 288, 289, 290, 291, 292, 293, 331, 332, 333, 334, 335, 336,
         337, 338, 339, 340, 341, 342, 343, 381, 382, 383, 384, 385, 386, 387,
         388, 389, 390, 391, 392, 393, 431, 432, 433, 434, 435, 436, 437, 438,
         439, 440, 441, 442, 443, 481, 482, 483, 484, 485, 486, 487, 488, 489,
         490, 491, 492, 493, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540,
         541, 542, 543]], device='cuda:0'), tensor([[ 11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  61,
          62,  63,  64,  65,  66,  67,  68,  

Get Embeddings

In [17]:
embedder =  PatchEmbed(sig_size=[12,5000], patch_size=[1,100], in_chans=1, embed_dim=768).to(device)
embedder(data).shape

torch.Size([1, 600, 768])

Get Models

In [36]:
from src.models.vision_transformer import vit_tiny, vit_predictor
vit = vit_tiny(sig_size=[12,5000], patch_size=[1,100], in_chans=1).to(device)

600
torch.Size([1, 600, 192])
torch.Size([1, 600, 192])


In [37]:
out = vit.forward(data)
print(out.shape, out)

torch.Size([1, 600, 192]) tensor([[[-1.0816, -0.9743, -1.2279,  ...,  1.0870,  1.2226,  1.2479],
         [ 0.6709,  0.3780, -0.0387,  ...,  1.0209,  1.2078,  1.2800],
         [ 0.9280,  0.8592,  0.6504,  ...,  0.9852,  1.1914,  1.2185],
         ...,
         [-0.2804,  0.8453,  0.1495,  ...,  1.1970,  1.0874,  1.3430],
         [-1.8311,  0.9788,  0.7349,  ...,  1.1908,  1.1223,  1.3676],
         [-2.1792,  0.1822,  0.6932,  ...,  1.1661,  1.2745,  1.2774]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)


In [50]:
pred = vit_predictor(num_patches=600,embed_dim=192).to(device)

In [51]:
print(out.shape)

torch.Size([1, 600, 192])


In [54]:
out2 = pred.forward(out, masks_enc, masks_pred)
print(out2.shape, out2)

predictor_pos_embed: torch.Size([1, 600, 384])
x_pos_embed torch.Size([1, 600, 384])
x: torch.Size([1, 600, 384])
m: torch.Size([1, 77])
mask_keep: torch.Size([1, 77, 384])
torch.Size([1, 77, 384])
temp: torch.Size([1, 77, 384])
x: torch.Size([1, 600, 384])
m: torch.Size([1, 77])
mask_keep: torch.Size([1, 77, 384])
torch.Size([1, 77, 384])


RuntimeError: The size of tensor a (600) must match the size of tensor b (77) at non-singleton dimension 1